In [1]:
from dbhelper import PgDB, pd_query
import pandas as pd

## Query for getting average for day of week for past year

In [8]:
munger_id = 4

In [12]:
query = '''
        select
            extract(dow from m.date),
            hour,
            minute,
            avg(metric) as waittime
        from mungedata m
        join datefeatures d
        on d.date = m.date
        where
            munger_id = %s
            and crossing_id = %s
            and extract(dow from m.date) = extract(dow from date '%s')
            and m.date >= date '%s' - interval '1 year' 
            and m.date < '%s'
            and is_waittime = true
        group by extract(dow from m.date), hour, minute
        order by extract(dow from m.date), hour, minute
        ;
        '''

xid = 1
date = '12/12/15'

df = pd_query(query % (munger_id, xid, date, date, date))

In [12]:
df.head()

,date_part,hour,minute,avg
0,6,0,0,3.181205
1,6,0,5,2.998544
2,6,0,10,2.865013
3,6,0,15,2.753212
4,6,0,20,2.653161


## Keep it simple!  Just create a baseline for average of 2014

In [13]:
query = '''
        select
            crossing_id,
            extract(dow from m.date) as dow,
            hour,
            minute,
            avg(metric) as waittime
        from mungedata m
        join datefeatures d
        on d.date = m.date
        where
            munger_id = {0}
            and m.date >= '2014-1-1'
            and m.date < '2015-1-1'
            and is_waittime = true
        group by crossing_id, extract(dow from m.date), hour, minute
        order by crossing_id, extract(dow from m.date), hour, minute
        ;
        '''

df = pd_query(query.format(munger_id))

In [14]:
from dbhelper import get_crossings
get_crossings()

,location_id,lane_id,direction_id,location_name,lane_name,direction_name
id,,,,,,
15,136,0,0,Lynden,Car,Northbound
16,136,0,1,Lynden,Car,Southbound
17,136,4,0,Lynden,Truck,Northbound
18,136,4,1,Lynden,Truck,Southbound
8,135,2,0,Pacific Highway,Bus,Northbound
7,135,2,1,Pacific Highway,Bus,Southbound
6,135,0,0,Pacific Highway,Car,Northbound
5,135,0,1,Pacific Highway,Car,Southbound
10,135,3,0,Pacific Highway,Fast,Northbound


### DOW definition
* Postgres DOW: 0=sunday
* Python DOW: 0=monday

In [19]:
pytopg = {6:0, 5:6, 4:5, 3:4, 2:3, 1:2, 0:1}

query = '''
        insert into predictions (model_version, munger_id, crossing_id, date, waittime)
        values('b2014', %s, %s, '%s', %s)
        '''

In [20]:
import datetime

with PgDB() as db:
    for xid in [2, 6]:
        dt = datetime.date(2015, 1, 1)

        while dt < datetime.date(2018, 1, 1):

            dow = pytopg[dt.weekday()]

            for row in df[(df.crossing_id == xid) & (df.dow == dow)].itertuples():
                db.cur.execute(query % (munger_id, xid, 
                                        str(datetime.datetime(dt.year, dt.month, dt.day, row[3], row[4], 0)), 
                                        row[5])
                              )
            
            dt = dt + datetime.timedelta(days=1)

            db.conn.commit()

